In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/S2SD')

#### * Make Dataset

In [1]:
import experiment
experiment.ExpA( experiment.auto_device ).create_ds(n_apps=60, n_paths=10, app_seed=1206, path_seed=1990,)

Create dataset @ __data__\X3e1c_A.npy
__data__\X3e1c_A.npy: 600


# Experiment

In [2]:
from experiment import ExpA as Exp
exp = Exp( Exp.auto_device ).load_ds(split_ratio=0.9)
exp

### Environments

In [3]:
tenv = exp.tEnv(seed=None, heed=None)
venv = exp.vEnv(heed=None)
venvs = exp.vEnvs(heeds=None)

# Train

### Policy

In [4]:
pie = exp.pie()
pie.Count()

1840516

In [ ]:
pie.Show()

### Training Loop

In [2]:
exp.list_Huristic()

['pie_Huristic_Edge',
 'pie_Huristic_Edge_Cloud',
 'pie_Huristic_Edge_Cloud_Limited',
 'pie_Huristic_Random',
 'pie_Huristic_Random_Edge',
 'pie_Huristic_Round_Robin_Edge',
 'pie_Huristic_Round_Robin_Edge_Cloud',
 'pie_Huristic_Zonal_Edge']

In [5]:
_ = exp.train(
    tenv,
    venvs,
    pie,
    learning_rate=      1e-5,
    lref=               0.05,
    weight_decay=       0.0,
    epochs =            500,
    batch_size =        16,
    learn_times =       50,
    tuf =               5,
    epsilon_range =     (1.0, 0.1),
    explore_pies =      ['pie_Huristic_Random', 'pie_Huristic_Edge_Cloud', 'pie_Huristic_Round_Robin_Edge_Cloud' ],
    explore_per_pie=    True,
    min_memory_pies=     exp.list_Huristic(),
    double =            False,
    validation_interval = 0.01,
    checkpoint_interval = 0.1,
    gradient_clipping = 1.0,
    save_at =           '/content/drive/MyDrive/Colab Notebooks/S2SD/__results__',
)

[*] Explored Min-Memory [1000] Steps, Memory Size is [1000]
 [Pre-Validation]
 => (MEAN) :: Return:-47.03623958333333, Steps:48.0
 => (SUM)  :: Return:-2822.1743749999996, Steps:2880.0
Checkpoint @ /content/drive/MyDrive/Colab Notebooks/S2SD/__results__\pre.pie



  1%|          | 5/500 [05:24<9:42:49, 70.65s/it]

 [Validation]
 => (MEAN) :: Return:-15.567352083333335, Steps:48.0
 => (SUM)  :: Return:-934.0411250000001, Steps:2880.0


  2%|▏         | 10/500 [11:09<10:06:19, 74.24s/it]

 [Validation]
 => (MEAN) :: Return:-14.329735416666669, Steps:48.0
 => (SUM)  :: Return:-859.7841250000001, Steps:2880.0


  3%|▎         | 13/500 [14:22<9:09:27, 67.69s/it] 

# Test

In [ ]:
import s2sdqn
import torch as tt
import torch.nn as nn
import os
import numpy as np
import matplotlib.pyplot as plt
import s2sdqn.ktf as ktf

### Policy

In [ ]:
pie = exp.pie()
pie.Count()

In [ ]:
pie.Show()

In [ ]:
load_name = 'final.pie'
load_from = '/content/drive/MyDrive/Colab Notebooks/S2SD/__results__'

pie.load(os.path.join(load_from, load_name))
pie.eval()


In [ ]:
results = s2sdqn.dqn.Eval.load_plot_training_result(os.path.join(load_from, 'results.npz'))

# Validation

In [ ]:
envs = [venv]
episodes=len(venv.frozen)

In [ ]:
envs =venvs
episodes=1

## DQN

In [ ]:
pie.theta.do_store_attention(True)

In [ ]:
mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
s2sdqn.dqn.Eval.validate_policy(
    pie=pie,
    envs=envs,
    episodes=episodes,
    episodic_verbose=0,
    verbose_result=True
)
print(f"""
{mean_return=}
{mean_steps=}
{sum_return=}
{sum_steps=}
{validate_acts=}
""")
v_dqn = validate_acts

In [ ]:
ktf.Aview.view_mh_attention_encoders(
    former=pie.theta,
    batch_index=None,
    **ktf.Aview.dca
)
ktf.Aview.view_attention_encoders(
    former=pie.theta,
    batch_index=None,
    **ktf.Aview.dca
)

In [ ]:
ktf.Aview.view_mh_attention_decoder(
    former=pie.theta,
    cross=False,
    batch_index=None,
    **ktf.Aview.dca
)
ktf.Aview.view_attention_decoder(
    former=pie.theta,
    cross=False,
    batch_index=None,
    **ktf.Aview.dca
)

In [ ]:
ktf.Aview.view_mh_attention_decoder(
    former=pie.theta,
    cross=True,
    batch_index=None,
    **ktf.Aview.dca
)
ktf.Aview.view_attention_decoder(
    former=pie.theta,
    cross=True,
    batch_index=None,
    **ktf.Aview.dca
)

In [ ]:
pie.theta.do_store_attention(False)

## Fixed

### Edge-0

In [ ]:
mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
s2sdqn.dqn.Eval.validate_policy(
    pie=s2sdqn.pie.FixedPie(Aseq=[0 for _ in range(venv.T)]),
    envs=envs,
    episodes=episodes,
    episodic_verbose=0,
    verbose_result=True
)
print(f"""
{mean_return=}
{mean_steps=}
{sum_return=}
{sum_steps=}
{validate_acts=}
""")

### Edge-1

In [ ]:
mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
s2sdqn.dqn.Eval.validate_policy(
    pie=s2sdqn.pie.FixedPie(Aseq=[1 for _ in range(venv.T)]),
    envs=envs,
    episodes=episodes,
    episodic_verbose=0,
    verbose_result=True
)
print(f"""
{mean_return=}
{mean_steps=}
{sum_return=}
{sum_steps=}
{validate_acts=}
""")

### Edge-2

In [ ]:
mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
s2sdqn.dqn.Eval.validate_policy(
    pie=s2sdqn.pie.FixedPie(Aseq=[2 for _ in range(venv.T)]),
    envs=envs,
    episodes=episodes,
    episodic_verbose=0,
    verbose_result=True
)
print(f"""
{mean_return=}
{mean_steps=}
{sum_return=}
{sum_steps=}
{validate_acts=}
""")

### Cloud

In [ ]:
mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
s2sdqn.dqn.Eval.validate_policy(
    pie=s2sdqn.pie.FixedPie(Aseq=[3 for _ in range(venv.T)]),
    envs=envs,
    episodes=episodes,
    episodic_verbose=0,
    verbose_result=True
)
print(f"""
{mean_return=}
{mean_steps=}
{sum_return=}
{sum_steps=}
{validate_acts=}
""")

## Random

### Full Random

In [ ]:
mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
s2sdqn.dqn.Eval.validate_policy(
    pie=s2sdqn.pie.RandomPie(Alow=0, Ahigh=venv.A),
    envs=envs,
    episodes=episodes,
    episodic_verbose=0,
    verbose_result=True
)
print(f"""
{mean_return=}
{mean_steps=}
{sum_return=}
{sum_steps=}
{validate_acts=}
""")

### Edge Random

In [ ]:
mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
s2sdqn.dqn.Eval.validate_policy(
    pie=s2sdqn.pie.RandomPie(Alow=0, Ahigh=venv.sim.E),
    envs=envs,
    episodes=episodes,
    episodic_verbose=0,
    verbose_result=True
)
print(f"""
{mean_return=}
{mean_steps=}
{sum_return=}
{sum_steps=}
{validate_acts=}
""")

## Hueristic

In [ ]:
exp.list_Huristic()

### pie_Huristic_Edge

In [ ]:
mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
s2sdqn.dqn.Eval.validate_policy(
    pie='pie_Huristic_Edge',
    envs=envs,
    episodes=episodes,
    episodic_verbose=0,
    verbose_result=True
)
print(f"""
{mean_return=}
{mean_steps=}
{sum_return=}
{sum_steps=}
{validate_acts=}
""")

### pie_Huristic_Edge_Cloud

In [ ]:
mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
s2sdqn.dqn.Eval.validate_policy(
    pie='pie_Huristic_Edge_Cloud',
    envs=envs,
    episodes=episodes,
    episodic_verbose=0,
    verbose_result=True
)
print(f"""
{mean_return=}
{mean_steps=}
{sum_return=}
{sum_steps=}
{validate_acts=}
""")

### pie_Huristic_Edge_Cloud_Limited

In [ ]:
mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
s2sdqn.dqn.Eval.validate_policy(
    pie='pie_Huristic_Edge_Cloud_Limited',
    envs=envs,
    episodes=episodes,
    episodic_verbose=0,
    verbose_result=True
)
print(f"""
{mean_return=}
{mean_steps=}
{sum_return=}
{sum_steps=}
{validate_acts=}
""")

### pie_Huristic_Random

In [ ]:
mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
s2sdqn.dqn.Eval.validate_policy(
    pie='pie_Huristic_Random',
    envs=envs,
    episodes=episodes,
    episodic_verbose=0,
    verbose_result=True
)
print(f"""
{mean_return=}
{mean_steps=}
{sum_return=}
{sum_steps=}
{validate_acts=}
""")

### pie_Huristic_Random_Edge

In [ ]:
mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
s2sdqn.dqn.Eval.validate_policy(
    pie='pie_Huristic_Random_Edge',
    envs=envs,
    episodes=episodes,
    episodic_verbose=0,
    verbose_result=True
)
print(f"""
{mean_return=}
{mean_steps=}
{sum_return=}
{sum_steps=}
{validate_acts=}
""")

### pie_Huristic_Round_Robin_Edge

In [ ]:
mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
s2sdqn.dqn.Eval.validate_policy(
    pie='pie_Huristic_Round_Robin_Edge',
    envs=envs,
    episodes=episodes,
    episodic_verbose=0,
    verbose_result=True
)
print(f"""
{mean_return=}
{mean_steps=}
{sum_return=}
{sum_steps=}
{validate_acts=}
""")

### pie_Huristic_Round_Robin_Edge_Cloud

In [ ]:
mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
s2sdqn.dqn.Eval.validate_policy(
    pie='pie_Huristic_Round_Robin_Edge_Cloud',
    envs=envs,
    episodes=episodes,
    episodic_verbose=0,
    verbose_result=True
)
print(f"""
{mean_return=}
{mean_steps=}
{sum_return=}
{sum_steps=}
{validate_acts=}
""")

### pie_Huristic_Zonal_Edge

In [ ]:
mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
s2sdqn.dqn.Eval.validate_policy(
    pie='pie_Huristic_Zonal_Edge',
    envs=envs,
    episodes=episodes,
    episodic_verbose=0,
    verbose_result=True
)
print(f"""
{mean_return=}
{mean_steps=}
{sum_return=}
{sum_steps=}
{validate_acts=}
""")

## Custom Action Sequence

In [ ]:
mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
s2sdqn.dqn.Eval.validate_policy(
    pie=s2sdqn.pie.FixedPie(Aseq=[0,1,2,3,0,1,2,3,0,1,2,3,0,1,2]),
    envs=envs,
    episodes=episodes,
    episodic_verbose=0,
    verbose_result=True
)
print(f"""
{mean_return=}
{mean_steps=}
{sum_return=}
{sum_steps=}
{validate_acts=}
""")

# FDA

In [ ]:
envs = venvs

In [ ]:
for fenv in envs:
    print(f'fenv: {fenv}')
    n_actions = fenv.A
    n_edges =   fenv.sim.E
    n_steps =   fenv.T
    #====================================================================================================
    #====================================================================================================

    def randflowF(): return s2sdqn.fda.random_flow(0, n_actions+1, n_steps)
    def flow2sol(x): return [ int(abs(i)) % n_actions for i in x ]
    def flowcost(x):
        actions = flow2sol(x)
        total_reward, steps, _ = s2sdqn.dqn.Eval.test_solution(fenv, actions, max_steps=None, verbose=0)
        return -total_reward

    #====================================================================================================
    #====================================================================================================


    #print(f'{placement=}')
    top_flows =[(np.zeros(n_steps) + j) for j in range(n_edges, n_actions)]

    Flow_X, Flow_fitness  = \
    s2sdqn.fda.optimize(
                    MAXITER=    50,
                    randflowF=  randflowF,
                    costF=      flowcost,
                    beta=       2,
                    alpha=      2,
                    base_flows= top_flows,
                    seed=       None)

    for i,(fx, fi) in enumerate(zip(Flow_X, Flow_fitness)):
        print(f'\n{i=}\n{flow2sol(fx)=}\n{fi=}\n')

    # select top flows to continue
    selected_flows=[]
    selected_fitness = []
    for flow,fcost in zip(Flow_X, Flow_fitness):
        sol = flow2sol(flow)
        if sol not in selected_flows:
            selected_flows.append(sol)
            selected_fitness.append(fcost)
    selected_n = len(selected_flows)

    sorted_cost = np.argsort(selected_fitness)
    top_flows = [(np.array(selected_flows[sorted_cost[top]]), selected_fitness[sorted_cost[top]]) for top in range(selected_n)]
    #top_costs = [selected_fitness[sorted_cost[top]] for top in range(min(n_top, selected_n))]


    print(top_flows)
    mean_return, mean_steps, sum_return, sum_steps, validate_acts = \
    s2sdqn.dqn.Eval.validate_policy(
        envs=[fenv],
        pie=s2sdqn.pie.FixedPie(Aseq=top_flows[0][0]),
        episodes=1,
        episodic_verbose=0,
        verbose_result=True
    )
    print(f"""
    {mean_return=}
    {mean_steps=}
    {sum_return=}
    {sum_steps=}
    {validate_acts=}
    """)




# End